# Tau_Mix Sensitivity Study 

In [ ]:
import sys
githubPath = r"K:/Dropbox (GaTech)/DOE/BatchPaSR"
sys.path.insert(0, githubPath)
import BatchPaSR as bp; 
import CanteraTools as ct_tools; 
import cantera as ct;
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
phi_global = 0.635
phi_main = 0.3719
tau_sec = 0.28
tau_global = 20
tau_mix = [0.005, 0.01, 0.05, 0.1, 0.3]
P = 25 * ct.one_atm
miliseconds = 0.001

## Run Main Burner

In [ ]:
#making figures
fig1, (Tvst) = plt.subplots()
plt.xlabel('Time (seconds)')
Tvst.set_title("Average Tempature over Time")
fig2, (COvt, NOvt) = plt.subplots(nrows=2, ncols=1)
plt.xlabel('Time (seconds)')
COvt.set_title('Average CO Concentration over time (uncorrected ppm)')
NOvt.set_title('Average NO Concentration over time (uncorrected ppm)')
# fig3, (NOvCO) = plt.subplots()
# NOvCO.set_title('Average NO vs CO Concentration (uncorrected ppm)')
# plt.xlabel('CO Concentration')
# plt.ylabel('NO Concentration')

## Run 5 Different Secondary Reactors 

In [ ]:
for x in range(0,5):
#note add somewhere on graph that tau_mix is in miliseconds
#add somewhere to include # of particles
    [mfm, mam, mfs, mas] = solvePhi_airSplit(phi_global, phi_main, 100, 1)
    [vit_reactor, main_burner_DF] = runMainBurner(phi_main, (tau_global-tau_sec)*milliseconds)
    vit_particle = Particle.fromGas(vit_reactor.thermo, particle_mass = mfm+mam)

    dt = miliseconds * 1e-3 # in milliseconds
    secondary_gas = ct.Solution('gri30.xml')
    secondary_gas.TPX = 300, 25*ct.one_atm, {'CH4':1}
    secondary_part = Particle.fromGas(secondary_gas, particle_mass=mfs+mas)

    bp = PaSBR(particle_list=[vit_particle, secondary_part], dt=dt, N_MAX=1000)
    states = ct.SolutionArray(bp.mean_gas, extra=['t'])
    enthalpy = []
    t = np.arange(0, 1*milliseconds, bp.dt)

    #bp.prepEntrainment(added_gas = vit_reactor.thermo, total_mass_added = (mam+mfm), tau_ent = totalTime, numParticles=20, method='constant')
    for i in range(0,t.size):
        #bp.entrain(t[i])
        bp.react()
        bp.mix(tau_mix=tau_mix[x]*milliseconds)
        states.append(bp.mean_gas.state, t=t[i])
        enthalpy.append(bp.mean_gas.enthalpy_mass)
    Tvst.plot(states.t, states.T, label = 'Tau_mix = %s' %(tau_mix[x]))
    COvt.plot(states.t, states('CO').X*1e6, label = 'Tau_mix = %s' %(tau_mix[x]))
    NOvt.plot(states.t, states('NO').X*1e6, label = 'Tau_mix = %s' %(tau_mix[x]))
    # NOvCO.plot(states('CO').X*1e6, states('NO').X*1e6, label = 'Tau_mix = %s' %(tau_mix[x]) )

## Plot Results 

In [ ]:

for ax in [COvt, NOvt, Tvst]:
    ax.grid(True)
    ax.label_outer()
    ax.legend()

fig1.savefig('AvTvst.png')
fig2.savefig('Entrained_NO_CO_Concentrations.png')

plt.show()